In [0]:

from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

In [0]:
source_view = "capstone.medisure.silver_claims_view"
source_delta = "capstone.medisure.silver_claims_delta_table"
target_table = "capstone.medisure.gold_claims_delta_table"

# Check if Delta source exists
delta_exists = spark.sql(f"""
    SELECT COUNT(*) 
    FROM system.information_schema.tables
    WHERE table_catalog = 'capstone'
      AND table_schema = 'medisure'
      AND table_name = 'silver_claims_delta_table'
""").collect()[0][0] > 0

# Pick source depending on existence
source_table = source_delta if delta_exists else source_view
print(f"✅ Using {source_table} as source for MERGE")

# Ensure target exists with correct schema (empty table)
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {target_table}
USING DELTA
AS SELECT * FROM {source_table} WHERE 1=0
""")

# Build update set dynamically
cols = spark.table(source_table).columns
set_expr = ", ".join([f"gold.{c} = silver.{c}" for c in cols if c not in ["ClaimID","MemberID","ProviderID"]])

# Merge statement
merge_sql = f"""
MERGE INTO {target_table} AS gold
USING {source_table} AS silver
ON gold.ClaimID = silver.ClaimID
   AND gold.MemberID = silver.MemberID
   AND gold.ProviderID = silver.ProviderID
WHEN MATCHED THEN UPDATE SET {set_expr}
WHEN NOT MATCHED THEN INSERT *
"""

spark.sql(merge_sql)
